In [18]:
import os
import streamlit as st
from datetime import datetime, timedelta
from typing import List, Dict
import requests
from langchain_core.documents import Document
from langchain_cohere import CohereEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import dotenv
import re

# Load environment variables
dotenv.load_dotenv()

# Set environment variables
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["COHERE_API_KEY"] = os.getenv("COHERE_API_KEY")
os.environ["NEWSAPI_KEY"] = os.getenv("NEWS_API_KEY")
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")

In [19]:
class NewsCollector:
    def __init__(self):
        self.base_url = "https://newsapi.org/v2/top-headlines"
        self.api_key = os.getenv("NEWSAPI_KEY")
        
    def get_news(self, country: str, category: str) -> List[Dict]:
        params = {
            "country": country,
            "category": category,
            "apiKey": self.api_key,
            "pageSize": 10
        }
        response = requests.get(self.base_url, params=params)
        #print(response.json())
        return response.json()["articles"]
    
    def preprocess_news(self, articles: List[Dict], region: str, category: str) -> List[Document]:
        documents = []
        for article in articles:
            if article['title'] != "[Removed]" and article['description'] != "[Removed]" and article['content'] != "[Removed]":
                content = f"Title: {article['title']}\nDescription: {article['description']}\nContent: {article['content']}"
                summary_content = f"Title: {article['title']}\nDescription: {article['description']}"
                metadata = {
                    "published_date": article["publishedAt"],
                    "category": category,
                    "region": region,
                    "source": article["source"]["name"],
                    "summary_content": summary_content  # Store concise version in metadata

                }
                documents.append(Document(page_content=content, metadata=metadata))
        return documents


In [20]:
region ="us"
category = "business"
news_collector = NewsCollector()
articles = news_collector.get_news(region, category)
documents = news_collector.preprocess_news(articles,region,category)

In [21]:
class VectorStore:
    def __init__(self):
        self.embeddings = CohereEmbeddings(
            model='embed-english-v3.0'
        )
        self.vectorstore = None
    
    def create_vectorstore(self, documents: List[Document]):
        self.vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=self.embeddings
        )
        return self.vectorstore

class NewsSummarizer:
    def __init__(self):
        self.llm = ChatGroq(model="llama3-8b-8192")
        
    def summarize_news(self, documents: List[Document], current_date: str) -> str:
        template = """Current date: {current_date}
        Based on the following news articles and their published dates, provide a comprehensive summary of the latest news:
        
        {context}
        
        Prioritize more recent news while maintaining coherence in the summary.
        """
        
        prompt = ChatPromptTemplate.from_template(template)
        
        chain = (
            prompt 
            | self.llm 
            | StrOutputParser()
        )
        
        context = "\n\n".join([
            f"Article ({doc.metadata['published_date']}):\n{doc.metadata['summary_content']}"
            for doc in documents
        ])
        
        return chain.invoke({
            "context": context,
            "current_date": current_date
        })

In [22]:
# Create vector store

vector_store = VectorStore()
vectorstore = vector_store.create_vectorstore(documents)

news_summarizer = NewsSummarizer()
# Generate summary
current_date = datetime.now().strftime("%Y-%m-%d")
summary = news_summarizer.summarize_news(documents, current_date)
print(summary)

Here is a comprehensive summary of the latest news, prioritizing more recent news and maintaining coherence:

The past week has been marked by significant developments in the world of finance, technology, and politics. Starting with the cryptocurrency market, BlackRock's accumulating Bitcoin holdings have signaled a bullish trend for the digital currency, sparking hopes of a price surge.

In the world of politics, the upcoming US presidential election has taken center stage, with Robinhood offering users the chance to trade contracts for Kamala Harris and Donald Trump starting Monday. Meanwhile, inflation, Big Tech earnings, and a crucial jobs report are set to shape the health of the US economy and stock market, which is trading near record highs.

In the tech space, Tesla's stock has boomed after a strong Q3, with Elon Musk predicting 20-30% sales growth next year. The company's stock surge has also led to comparisons with Warren Buffett, with Musk channeling the legendary investor.


In [23]:
class CustomNewsRetriever:
    def __init__(self, vectorstore):
        self.vectorstore = vectorstore
        self.llm = ChatGroq(model="llama3-8b-8192")
    
    def generate_questions(self, user_query: str) -> List[str]:
        template = """You are an AI language model assistant. Your task is to generate 5 different sub questions OR alternate versions of the given user question to retrieve relevant documents from a vector database.

        By generating multiple versions of the user question,
        your goal is to help the user overcome some of the limitations
        of distance-based similarity search.

        By generating sub questions, you can break down questions that refer to multiple concepts into distinct questions. This will help you get the relevant documents for constructing a final answer

        If multiple concepts are present in the question, you should break into sub questions, with one question for each concept

        Provide these alternative questions separated by newlines between XML tags. For example:

        <questions>
        - Question 1
        - Question 2
        - Question 3
        </questions>

        Original question: {question}"""
        
        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | self.llm | StrOutputParser()
        
        result = chain.invoke({"question": user_query})
        result = re.search(r'<questions>(.*?)</questions>', result, re.DOTALL).group(1)
        return result
    
    def deduplicate_docs(self, docs: List[Document]) -> List[Document]:
        """
        Deduplicate documents based on their content and metadata
        """
        unique_docs = []
        seen_contents = set()
        
        for doc in docs:
            # Create a unique identifier using content and published date
            content_identifier = (
                doc.page_content,
                doc.metadata.get('published_date', '')
            )
            
            if content_identifier not in seen_contents:
                seen_contents.add(content_identifier)
                unique_docs.append(doc)
        
        return unique_docs
    
    def get_relevant_docs(self, questions: List[str]) -> List[Document]:
        all_docs = []
        for question in questions:
            docs = self.vectorstore.similarity_search(question)
            all_docs.extend(docs)
        
        # Deduplicate documents
        return self.deduplicate_docs(all_docs)
    
    def answer_query(self, user_query: str, docs: List[Document], current_date: str) -> str:
        template = """Current date: {current_date}
        Based on the following news articles and their published dates, answer this question: {question}
        
        Context:
        {context}
        
        Provide a clear and focused answer based on the most recent and relevant information.
        """
        
        prompt = ChatPromptTemplate.from_template(template)
        chain = prompt | self.llm | StrOutputParser()
        
        # Use full content for detailed answers to specific queries
        context = "\n\n".join([
            f"Article ({doc.metadata['published_date']}):\n{doc.page_content}"
            for doc in docs
        ])
        
        return chain.invoke({
            "context": context,
            "question": user_query,
            "current_date": current_date
        })

In [24]:
current_date = datetime.now().strftime("%Y-%m-%d")

user_query = "What is the latest news on business in the US?"

custom_retriever = CustomNewsRetriever(vector_store.vectorstore)

questions = custom_retriever.generate_questions(user_query)
print(questions)

relevant_docs = custom_retriever.get_relevant_docs(questions)
print(relevant_docs)

answer = custom_retriever.answer_query(user_query, relevant_docs, current_date)
print(answer)


- What is the latest news on US business trends?
- What are the recent developments in the US economy?
- What is the current state of the US stock market?
- What are the latest updates on US business leaders and executives?
- What are the most significant business stories in the US this week?



Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com with any questions".
Retrying langchain_cohere.embeddings.CohereEmbeddings.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised TooManyRequestsError: status_code: 429, body: data=None message="You are using a Trial key, which is limited to 100 API calls / minute. You can continue to use the Trial key for free or upgrade to a Production key with higher rate limits at 'https://dashboard.cohere.com/api-keys'. Contact us on 'https://discord.gg/XW44jPfYJu' or email us at support@cohere.com wit

KeyboardInterrupt: 